In [1]:
#importing libraries
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = '508d7b2a827c44b5849ca244c4b88bbc',
                                                             client_secret ='7fd8452e94824d518c0c19a8e4f737d7'))
sp

In [2]:
#function to get the audio features (acousticness, loudness, energy, etc.) of each song from a playlist  

def get_audio_features(playlist_code):
    
    track_id = [] #list to store the track ID's of each song of the playlist

    for count in range(0, 10000, 50):
        playlist = sp.user_playlist_tracks(user="spotify",
                                       playlist_id=playlist_code,
                                       limit=50,
                                       offset=count)
    
        for item in playlist['items']:
            # Check if 'track' key exists and if it's not None
            if 'track' in item and item['track'] is not None:
                track = item['track']
            # Check if 'id' key exists and if it's not None
                if 'id' in track and track['id'] is not None:
                    track_id.append(track['id'])

    chunk_size = 50  #partitioning in sets of 50 otherwise the spotipy rate limit is exceeded
    chunks = [track_id[i:i + chunk_size] for i in range(0, len(track_id), chunk_size)]

    big_playlist_features = [] #list to store the audio features of each song

    for chunk in chunks:
    # Retrieve audio features for the current chunk of track IDs
        features = sp.audio_features(chunk)
        big_playlist_features.extend(features)
    
    features_df = pd.DataFrame(data=big_playlist_features, columns=big_playlist_features[0].keys())
    return features_df

In [3]:
#function to get the song titles, album, artist, duration, popularity and reference of each song from a playlist 

def get_song_names(playlist_code):
    
    track_id = []
    track_name = []
    track_artists = []
    track_album = []
    track_duration = []
    track_href = []
    track_pop = []

    for count in range(0, 10000, 50):
        playlist = sp.user_playlist_tracks(user="spotify",
                                       playlist_id="1G8IpkZKobrIlXcVPoSIuf",
                                       limit=50,
                                       offset=count)
        
        for item in playlist['items']:
            track = item.get('track')  # use .get() method to safely access 'track' and handle None
            if track is not None:
                track_id.append(track.get('id', ''))
                track_name.append(track.get('name', ''))
                track_artists.append(', '.join([artist['name'] for artist in track.get('artists', [])]))
                track_album.append(track.get('album', {}).get('name', ''))
                track_duration.append(track.get('duration_ms', 0) / 1000)  # Convert milliseconds to seconds
                track_href.append(track.get('href', ''))
                track_pop.append(track.get('popularity', 0))

# create DataFrame
    playlist_df = pd.DataFrame({'id': track_id,
                             'song_name': track_name,
                             'artist': track_artists,
                             'album': track_album,
                             'duration_sec': track_duration,
                             'h_ref': track_href,
                             'popularity': track_pop})
    return playlist_df

In [5]:
#function to combine the song titles and the audio features of each song from a playlist 

def get_song_details(playlist_codes):
    # Initialize an empty list to store the DataFrames for each playlist
    dfs = []
    
    # Iterate through each playlist code
    for code in playlist_codes:
        # Fetch song names and audio features for the current playlist code
        song_names_df = get_song_names(code)
        audio_features_df = get_audio_features(code)
        
        # Merge the dataframes based on 'id'
        df = pd.merge(song_names_df, audio_features_df, on='id', how='right')
        
        # Append the current playlist's details to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list along the rows (axis=0)
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df = combined_df.dropna()
    combined_df = combined_df.drop_duplicates()
    
    return combined_df

In [ ]:
#list of playlists
playlists = ["4hOKQuZbraPDIfaGbM3lKI","1G8IpkZKobrIlXcVPoSIuf"]

In [6]:
df = get_song_details(playlists)

In [7]:
df #final output

,id,song_name,artist,album,duration_sec,h_ref,popularity,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
2,7qiZfU4dY1lWllzX7mPBI3,Shape of You,Ed Sheeran,÷ (Deluxe),233.712,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,89.0,0.825,0.652,1,...,0.000000,0.0931,0.931,95.977,audio_features,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
3,7KXjTSCq5nL1LoYtL7XAwS,HUMBLE.,Kendrick Lamar,DAMN.,177.000,https://api.spotify.com/v1/tracks/7KXjTSCq5nL1...,88.0,0.908,0.621,1,...,0.000054,0.0958,0.421,150.011,audio_features,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,https://api.spotify.com/v1/tracks/7KXjTSCq5nL1...,https://api.spotify.com/v1/audio-analysis/7KXj...,177000,4
16,5Ohxk2dO5COHF1krpoPigN,Sign of the Times,Harry Styles,Harry Styles,340.706,https://api.spotify.com/v1/tracks/5Ohxk2dO5COH...,86.0,0.516,0.595,5,...,0.000000,0.1090,0.222,119.972,audio_features,spotify:track:5Ohxk2dO5COHF1krpoPigN,https://api.spotify.com/v1/tracks/5Ohxk2dO5COH...,https://api.spotify.com/v1/audio-analysis/5Ohx...,340707,4
21,6HZILIRieu8S0iqY8kIKhj,DNA.,Kendrick Lamar,DAMN.,185.946,https://api.spotify.com/v1/tracks/6HZILIRieu8S...,83.0,0.638,0.523,1,...,0.000000,0.0842,0.422,139.913,audio_features,spotify:track:6HZILIRieu8S0iqY8kIKhj,https://api.spotify.com/v1/tracks/6HZILIRieu8S...,https://api.spotify.com/v1/audio-analysis/6HZI...,185947,4
73,4Km5HrUvYTaSUfiSGPJeQR,Bad and Boujee (feat. Lil Uzi Vert),"Migos, Lil Uzi Vert",Culture,343.150,https://api.spotify.com/v1/tracks/4Km5HrUvYTaS...,2.0,0.926,0.666,11,...,0.000000,0.1230,0.168,127.079,audio_features,spotify:track:4Km5HrUvYTaSUfiSGPJeQR,https://api.spotify.com/v1/tracks/4Km5HrUvYTaS...,https://api.spotify.com/v1/audio-analysis/4Km5...,343150,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10042,2QSD3K3b3BJ8DPhGhQfDPW,Into The Valley,Skids,Scared To Dance,199.466,https://api.spotify.com/v1/tracks/2QSD3K3b3BJ8...,46.0,0.332,0.707,7,...,0.006120,0.1100,0.652,144.815,audio_features,spotify:track:2QSD3K3b3BJ8DPhGhQfDPW,https://api.spotify.com/v1/tracks/2QSD3K3b3BJ8...,https://api.spotify.com/v1/audio-analysis/2QSD...,199467,4
10043,49XnDVsYOHgV4gFZeCojKj,Tonight's Da Night,Redman,Whut? The Album,201.800,https://api.spotify.com/v1/tracks/49XnDVsYOHgV...,0.0,0.464,0.749,6,...,0.000046,0.3510,0.879,181.121,audio_features,spotify:track:49XnDVsYOHgV4gFZeCojKj,https://api.spotify.com/v1/tracks/49XnDVsYOHgV...,https://api.spotify.com/v1/audio-analysis/49Xn...,201800,4
10044,5Y9IIH8Xmo1nuk0gfFjc4Q,Figure 8,FKA twigs,M3LL155X,183.040,https://api.spotify.com/v1/tracks/5Y9IIH8Xmo1n...,0.0,0.694,0.710,2,...,0.001350,0.0697,0.415,119.964,audio_features,spotify:track:5Y9IIH8Xmo1nuk0gfFjc4Q,https://api.spotify.com/v1/tracks/5Y9IIH8Xmo1n...,https://api.spotify.com/v1/audio-analysis/5Y9I...,183040,4
10045,0ya0JYEFoXNviB8RMeHDtW,Like An Angel,The Mighty Lemon Drops,Rollercoaster - The Best Of 1986-1989,222.160,https://api.spotify.com/v1/tracks/0ya0JYEFoXNv...,0.0,0.332,0.800,1,...,0.001600,0.0850,0.832,149.240,audio_features,spotify:track:0ya0JYEFoXNviB8RMeHDtW,https://api.spotify.com/v1/tracks/0ya0JYEFoXNv...,https://api.spotify.com/v1/audio-analysis/0ya0...,222160,4
